In [1]:
import numpy as np
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
def plot_heatmap(Pq2_path, U):
    file = h5py.File(Pq2_path)
    data = np.array(file["Pmn2_avg"])
    U_index = list(file["U"]).index(U)

    plt.imshow(np.transpose(np.log10(abs(data[U_index,::-1,::-1]))), vmin=-5, vmax=0, origin='lower')
    plt.colorbar()
    plt.xlabel("$N_{A\\downarrow}$")
    plt.ylabel("$N_{A\\uparrow}$")
    plt.title("$U={}$".format(U))
    plt.show()

In [3]:
def two_site_etgent(U):
    S = np.sqrt(U**2 + 16)
    C = 8/(16+(U+S)**2)
    D = (U+S)/4

    return -np.log(2*C**2 * (1+D**4))

two_site_etgent = np.vectorize(two_site_etgent)

In [4]:
def two_site_n_dist(U, n):
    S = np.sqrt(U**2 + 16)
    C = 8/(16+(U+S)**2)
    D = (U+S)/4

    if n == 0:
        return 1/(2*(1+D**4))
    elif n == 1:
        return D**4/(1+D**4)
    elif n == 2:
        return 1/(2*(1+D**4))
    

two_site_n_dist = np.vectorize(two_site_n_dist)

In [5]:
def two_site_m_dist(U, m):
    S = np.sqrt(U**2 + 16)
    C = 8/(16+(U+S)**2)
    D = (U+S)/4

    if m == -1:
        return D**4/(2*(1+D**4))
    elif m == 0:
        return 1/(1+D**4)
    elif m == 1:
        return D**4/(2*(1+D**4))
    
two_site_m_dist = np.vectorize(two_site_m_dist)

In [6]:
def plot_distribution_two_site(Pq2_path, U):
    file = h5py.File(Pq2_path)
    avg_n = np.array(file["Pn2_avg"])
    avg_m = np.array(file["Pm2_avg"])
    err_n = np.array(file["Pn2_err"])
    err_m = np.array(file["Pm2_err"])
    U_index = list(file["U"]).index(U)

    n_list = range(len(avg_n[U_index, :]))
    m_list = range(-int((len(avg_m[U_index, :])-1)/2), int((len(avg_m[U_index, :])-1)/2 + 1))

    n_exact = two_site_n_dist(U, np.array(n_list))
    m_exact = two_site_m_dist(U, np.array(m_list))

    plt.errorbar(n_list, np.flip(avg_n[U_index, :]), yerr=err_n[U_index, :], label="$P_{n,2}$")
    plt.plot(n_list, n_exact, label="$P_{n,2}$ exact")
    plt.errorbar(m_list, avg_m[U_index, :], yerr=err_m[U_index, :], label="$P_{m,2}$")
    plt.plot(m_list, m_exact, label="$P_{m,2}$ exact")
    plt.ylim(top=1.0,bottom=0.0)
    plt.xlabel("$q_A$")
    plt.ylabel("P")
    plt.legend()
    plt.title("$U={}$".format(U))
    plt.show()

In [13]:
def plot_etgent_two_site(EtgEnt_path):
    file_etgent = h5py.File(EtgEnt_path)
    U_list = np.array(file_etgent["U"])

    avg_etgent = np.array(file_etgent["S2_avg"])
    err_etgent = np.array(file_etgent["S2_err"])

    exact_etgent = two_site_etgent(U_list)

    plt.errorbar(U_list, avg_etgent, yerr=err_etgent, label="$S_2$", linestyle='dashed', linewidth=2.0)
    plt.plot(U_list, exact_etgent)
    plt.xlim(left=min(U_list), right=max(U_list))
    plt.xlabel("$|U|$")
    plt.ylabel("Entropy")
    plt.legend()
    plt.title("Entanglement Entropy for the Attractive Hubbard Model")
    plt.show()